In [1]:
import pandas as pd
import dtale
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df=pd.read_csv(r'C:\Users\Pradeep Sharma\OneDrive\Desktop\zycus assignement\RestoInfo.csv')
d1=pd.DataFrame(df['reviews_list'].str.split(expand=True))
d2=d1.iloc[:,1:2]
d2.columns=['reviews_rating']
d3=d2.replace(',','',regex=True)

In [3]:
df.shape

(2069, 15)

In [4]:
dtale.show(df)

2021-07-05 11:46:45,994 - INFO     - NumExpr defaulting to 8 threads.


In [5]:
df.columns

Index(['Unnamed: 0', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [6]:
def preprocess(tweet):
   
   # Removing handles and hashtags
   tweet = re.sub('@[^\s]+','',str(tweet))
   tweet = re.sub(r'#', '', str(tweet))
   # Removing URLS
   tweet= re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',str(tweet))
   #removing all punchuation and special character
   tweet = re.sub('[^a-zA-Z0-9]',' ',str(tweet))
   #removing extra white space
   tweet = re.sub('[\s]+', ' ', str(tweet))
   tweet = re.sub('[\n]+', ' ', str(tweet))
   tweet=tweet.lower()
   return tweet
df['name']= df['name'].apply(lambda x:preprocess(x))
df.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   name                         2069 non-null   object
 1   online_order                 2069 non-null   object
 2   book_table                   2069 non-null   object
 3   rate                         1770 non-null   object
 4   votes                        2069 non-null   int64 
 5   location                     2069 non-null   object
 6   rest_type                    2052 non-null   object
 7   dish_liked                   962 non-null    object
 8   cuisines                     2069 non-null   object
 9   approx_cost(for two people)  2053 non-null   object
 10  reviews_list                 2069 non-null   object
 11  menu_item                    2069 non-null   object
 12  listed_in(type)              2069 non-null   object
 13  listed_in(city)              2069

In [8]:
((df.isnull().sum())/2069)*100

name                            0.000000
online_order                    0.000000
book_table                      0.000000
rate                           14.451426
votes                           0.000000
location                        0.000000
rest_type                       0.821653
dish_liked                     53.504108
cuisines                        0.000000
approx_cost(for two people)     0.773320
reviews_list                    0.000000
menu_item                       0.000000
listed_in(type)                 0.000000
listed_in(city)                 0.000000
dtype: float64

In [9]:
def preprocess(tweet):
   
   # Removing handles and hashtags
   tweet = re.sub('@[^\s]+','',str(tweet))
   tweet = re.sub(r'#', '', str(tweet))
   # Removing URLS
   tweet= re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',str(tweet))
   #removing all punchuation and special character
   tweet = re.sub('[^a-zA-Z0-9.]',' ',str(tweet))
   #removing extra white space
   tweet = re.sub('[\s]+', ' ', str(tweet))
   tweet = re.sub('[\n]+', ' ', str(tweet))
   tweet=tweet.lower()
   return tweet
d3['reviews_rating']= d3['reviews_rating'].apply(lambda x:preprocess(x))

In [10]:
d4=d3.replace('none','3.058241')
d4['reviews_rating']=d4['reviews_rating'].astype('float')

In [11]:
df_new=pd.concat([df,d4],axis=1)
df=df_new
num = df_new['rate'].mode()[0]
df['rate'].fillna(num, inplace=True)
num = df['rest_type'].mode()[0]
df['rest_type'].fillna(num, inplace=True)
num = df['dish_liked'].mode()[0]
df['dish_liked'].fillna(num, inplace=True)
num = df['approx_cost(for two people)'].mode()[0]
df['approx_cost(for two people)'].fillna(num, inplace=True)
num = df['reviews_rating'].mean()
df['reviews_rating'].fillna(num, inplace=True)

In [12]:
df.isnull().sum()

name                           0
online_order                   0
book_table                     0
rate                           0
votes                          0
location                       0
rest_type                      0
dish_liked                     0
cuisines                       0
approx_cost(for two people)    0
reviews_list                   0
menu_item                      0
listed_in(type)                0
listed_in(city)                0
reviews_rating                 0
dtype: int64

In [13]:
def preprocess(tweet):
   
   # Removing handles and hashtags
   tweet = re.sub('@[^\s]+','',str(tweet))
   tweet = re.sub(r'#', '', str(tweet))
   # Removing URLS
   tweet= re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',str(tweet))
   #removing all punchuation and special character
   tweet = re.sub('[^a-zA-Z]',' ',str(tweet))
   #removing extra white space
   tweet = re.sub('[\s]+', ' ', str(tweet))
   tweet = re.sub('[\n]+', ' ', str(tweet))
   tweet=tweet.lower()
   return tweet
train_data= df['reviews_list'].apply(lambda x:preprocess(x))
type(train_data)

pandas.core.series.Series

In [14]:

train_data=pd.DataFrame(train_data)
train_data.columns=['reviews_list']

In [15]:
train_data.reset_index(drop=True,inplace=True)

In [16]:
train_data.columns

Index(['reviews_list'], dtype='object')

In [17]:

def token(tweet):
  tweet = word_tokenize(tweet)
  return tweet
train_data['reviews_list']=train_data['reviews_list'].apply(lambda x:token(x))

In [18]:
#Performing Text Normalization
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
lem = WordNetLemmatizer()
def normalize(tweet):
  tweet = [lem.lemmatize(word) for word in tweet if not word in stop_words  ]
  #print(tweet)
  tweet = ' '.join(tweet)
  return tweet
train_data['reviews_list']= train_data['reviews_list'].apply(lambda x :normalize(x))
train_data.dropna(inplace=True)

In [19]:
x_tf=train_data['reviews_list']

In [20]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(r'D:\nlp\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)

2021-07-05 11:47:48,970 - INFO     - loading projection weights from D:\nlp\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin
2021-07-05 11:48:32,201 - INFO     - loaded (3000000, 300) matrix from D:\nlp\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin


In [21]:
tfidf = TfidfVectorizer()
tfidf.fit(x_tf)
dictionary = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))

In [22]:
def word2vectransform(list_of_sentance):
    zero = np.zeros(300)
    tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
    for sent in list_of_sentance: # for each review/sentence 
        sent = sent.split()
        sent_vec = np.zeros(300) # as word vectors are of zero length
        weight_sum =0; # num of words with a valid vector in the sentence/review
        for word in sent: # for each word in a review/sentence
            try:
                vec = model[word]
            except:
                vec = zero
            try:
                tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            except:
                tf_idf = 0
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        if weight_sum != 0:
                sent_vec /= weight_sum
        tfidf_sent_vectors.append(sent_vec)
        
    return tfidf_sent_vectors
df1 = word2vectransform(x_tf)

In [23]:
len(df1)

2069

In [24]:
df1=pd.DataFrame(df1)
df1

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.072529,0.047825,0.011798,0.133822,-0.047818,-0.019448,0.014258,-0.080753,0.010890,0.105555,...,-0.067812,0.050370,-0.070112,0.054002,-0.013745,-0.014909,-0.045540,0.011155,0.051515,0.010252
2,-0.119162,0.073029,0.033173,0.048827,0.042410,-0.015047,0.074423,-0.221287,-0.076278,0.119237,...,-0.008733,-0.071760,-0.082777,-0.015559,0.042735,-0.053695,0.039493,0.025939,0.011300,-0.195951
3,-0.034541,0.065107,0.021918,0.124551,-0.033969,0.006640,0.050304,-0.081844,0.016918,0.101676,...,-0.074636,0.027196,-0.090909,0.038695,-0.000346,-0.000446,0.002756,-0.028221,0.028660,0.005020
4,-0.044710,0.073162,0.036284,0.212046,-0.043609,0.042460,0.048763,-0.030578,-0.091182,0.170342,...,-0.036246,-0.048008,-0.111190,0.048997,0.018345,-0.033429,-0.028313,-0.014188,0.022996,0.044446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,-0.057145,0.058646,0.065195,0.212080,-0.053425,0.021283,0.040770,-0.098434,-0.045234,0.123160,...,-0.035575,-0.035280,-0.087135,0.027547,0.028147,-0.028573,-0.030742,-0.028719,-0.014205,0.073137
2065,-0.074874,0.063270,0.027750,0.199753,-0.032142,0.044201,0.044567,-0.033697,-0.031780,0.118164,...,-0.041637,-0.005149,-0.080122,0.062266,0.038989,-0.032153,-0.039832,-0.021534,0.059762,0.042794
2066,-0.021387,0.051862,-0.003174,0.119014,-0.067565,0.002005,0.062484,-0.098077,0.057358,0.104742,...,-0.058405,0.028756,-0.088965,0.024530,-0.033231,-0.017273,0.049298,-0.051270,0.040080,-0.030785
2067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
df.columns

Index(['name', 'online_order', 'book_table', 'rate', 'votes', 'location',
       'rest_type', 'dish_liked', 'cuisines', 'approx_cost(for two people)',
       'reviews_list', 'menu_item', 'listed_in(type)', 'listed_in(city)',
       'reviews_rating'],
      dtype='object')

In [26]:
dfx=df[['online_order', 'book_table', 'rate', 'votes', 'location',
       'rest_type', 'dish_liked', 'cuisines', 'approx_cost(for two people)',
        'menu_item', 'listed_in(type)', 'listed_in(city)']]

In [27]:
import pandas
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
dfx1=dfx.apply(le.fit_transform)

In [28]:
dfx1

,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),menu_item,listed_in(type),listed_in(city)
0,0,0,54,0,25,52,38,363,23,482,4,25
1,0,0,19,18,42,52,38,726,23,482,4,16
2,1,0,38,125,0,27,390,423,24,310,2,13
3,0,0,41,565,0,22,536,521,29,482,2,19
4,1,1,36,203,53,18,11,536,42,482,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...
2064,1,0,34,196,44,18,665,610,39,482,2,0
2065,1,1,38,458,25,18,211,521,3,482,2,11
2066,0,1,43,531,15,51,171,330,10,482,6,25
2067,0,0,54,0,0,52,38,722,29,482,4,0


In [29]:
df_final=pd.concat([df1,dfx1,df['name'],df['reviews_rating']],axis=1)
df_final_temp=pd.concat([df1,dfx1],axis=1)
df_final_temp1=df_final_temp.to_numpy()

In [30]:
rest_recom=df_final.pivot_table(index='name',columns=['cuisines','approx_cost(for two people)','rest_type','location','listed_in(city)'],values='reviews_rating').fillna(0)
rest_recom

cuisines                               0    1              2    3              \
approx_cost(for two people)             35   32             10   23             
rest_type                               52   52             20   52             
location                                68   3              49   0    17   74   
listed_in(city)                         4    0    2    19   21   0    8    11   
name                                                                            
1131 bar kitchen                       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1441 pizzeria                          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1522 the pub                           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1947                                   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1q1                                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                                    ...  ...  ...  ...  ...  ...  ...  ...   
zeeshan                                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
zengi pub restaurant                   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
ziyan s cafe 27th                      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
zodiac fortune select jp cosmos hotel  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
zodiac grills                          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

cuisines                               4    5    ...  765  766  767  768  769  \
approx_cost(for two people)             8    11  ...   35   35   29   25   29   
rest_type                               18   51  ...   18   18   52   27   39   
location                                25   41  ...   42   44   27   42   42   
listed_in(city)                         25   16  ...   17   16   0    0    19   
name                                             ...                            
1131 bar kitchen                       0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
1441 pizzeria                          0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
1522 the pub                           0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
1947                                   0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
1q1                                    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
...                                    ...  ...  ...  ...  ...  ...  ...  ...   
zeeshan                                0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
zengi pub restaurant                   0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
ziyan s cafe 27th                      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
zodiac fortune select jp cosmos hotel  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
zodiac grills                          0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   

cuisines                                         770            
approx_cost(for two people)                       0         39  
rest_type                               52        18        18  
location                                0         41        1   
listed_in(city)                         0    12   16   17   13  
name                                                            
1131 bar kitchen                       0.0  0.0  0.0  0.0  0.0  
1441 pizzeria                          0.0  0.0  0.0  0.0  0.0  
1522 the pub                           0.0  0.0  0.0  0.0  0.0  
1947                                   0.0  0.0  0.0  0.0  0.0  
1q1                                    0.0  0.0  0.0  0.0  0.0  
...                                    ...  ...  ...  ...  ...  
zeeshan                                0.0  0.0  0.0  0.0  0.0  
zengi pub restaurant                   0.0  0.0  0.0  0.0  0.0  
ziyan s cafe 27th                      0.0  0.0  0.0  0.0  0.0  
zodiac fortune select jp cosmos hotel  0.0  0.0  0.0  0.0  0.0  
zodiac grills                          0.0  0.0  0.0  0.0  0.0  

[1663 rows x 2014 columns]

In [31]:
from sklearn.metrics.pairwise import sigmoid_kernel
ss=sigmoid_kernel(rest_recom.values,rest_recom.values)
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(ss)

NearestNeighbors(algorithm='brute', metric='cosine')

In [32]:
ss1=pd.DataFrame(ss)

In [33]:
dr=df[['name']]
dr.reset_index(drop=False,inplace=True)

In [34]:
dr

,index,name
0,0,unique brew cafe resto
1,1,jayanthi sagar
2,2,rock stone ice cream factory
3,3,punjabi by nature 2 0
4,4,rayalaseema chefs
...,...,...
2064,2064,zeeshan
2065,2065,zaica dine wine
2066,2066,murphy s brewhouse the paul bangalore
2067,2067,cee yem


In [35]:
ind=pd.Series(dr.index,index=dr['name']).drop_duplicates()

In [36]:
ind=pd.DataFrame(ind)
ind['name']=ind.index
ind=pd.DataFrame(ind)
ind.columns=['index','name']


In [37]:
ind.reset_index(drop=True,inplace=True)
distances, indices = model_knn.kneighbors(ss1.loc[9].values.reshape(1, -1), n_neighbors = 5)
dt=indices.reshape(-1, 1)
dt=pd.DataFrame(dt)
dt.columns=['index']
df_rest=pd.merge(dt,ind,on='index',how='inner')
df_rest

,index,name
0,9,vinaya coffee moments
1,1534,food treat
2,1117,hyderabadi bawarchi biriyani
3,595,sathkar refershments
4,171,bang the ritz carlton


In [38]:
df_rest.name

0           vinaya coffee moments
1                      food treat
2    hyderabadi bawarchi biriyani
3            sathkar refershments
4           bang the ritz carlton
Name: name, dtype: object